In [ ]:
!pip install fiftyone qwen-vl-utils accelerate

# Using GUI Actor in FiftyOne

# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/visual_agents_workshop/blob/main/session_3/Testing_GUI_Actor_3B_on_FiftyOne_App_Dataset.ipynb)
# 


This tutorial demonstrates how to use the GUI-Actor vision-language models with FiftyOne for various visual understanding tasks.

Note: This is the model that we are going to fine-tune. First, we will see it's performance without fine-tuning.


## Load a Sample Dataset

First, let's load a UI dataset from the [FiftyOne Hugging Face Org](https://huggingface.co/Voxel51).

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "harpreetsahota/FiftyOne-GUI-Grounding-Eval",
    overwrite=True
    )

Launch the FiftyOne App to visualize the dataset (optional)

In [ ]:
fo.launch_app(dataset)

Take a look at the first sample and instruction to get a sense of what is in the dataset:

In [ ]:
from PIL import Image

Image.open(dataset.first().filepath)

In [ ]:
dataset.first().task

## Set Up GUI-Actor Integration

Register the GUI-Actor remote zoo model source and load the model.

In [ ]:
import fiftyone.zoo as foz

# Register the model source
foz.register_zoo_model_source("https://github.com/harpreetsahota204/gui_actor", overwrite=True)

# Load the `GUI-Actor-3B-Qwen2.5-VL` model

You can also use `GUI-Actor-7B-Qwen2.5-VL`

In [ ]:
model = foz.load_zoo_model(
    "microsoft/GUI-Actor-3B-Qwen2.5-VL",
    # install_requirements=True, #you can pass this to make sure you have all reqs installed
    )

In [ ]:
dataset.apply_model(
    model,
    prompt_field="task",
    label_field="guiactor_output"
    )

#### Adding Attention Heatmaps to Dataset

Since FiftyOne models can only return one prediction type (`keypoints` in our case), we save the attention heatmaps as PNG files during inference and need to manually link them back to the dataset.

This allows us to visualize both the predicted interaction points and the model's attention patterns together in the FiftyOne app.

In [ ]:
from pathlib import Path

for sample in dataset:
    # Generate expected heatmap path
    original_path = Path(sample.filepath)
    heatmap_path = original_path.parent / f"{original_path.stem}_attention.png"

    if heatmap_path.exists():
        # Add heatmap field using the saved PNG file
        sample["attention_heatmap"] = fo.Heatmap(map_path=str(heatmap_path))
        sample.save()
    else:
        print(f"Heatmap not found for {sample.filepath}")

Take a look at the first Sample to ensure we have parsed the output:

In [ ]:
dataset.first()

In [ ]:
# Visualize all results in the FiftyOne App
session = fo.launch_app(dataset)

In [ ]:
session.freeze()